In [1]:
"""
This file defines several useful custom layers (these are not actual keras.layers.Layer objects) to be used in the network
You can call them like this:
    `LayerName(...)(x)`

This is useful since very often you need to chain some layers (like the classical Conv + BatchNorm + NonLinearity)
"""

import tensorflow as tf
from keras.layers import Dense as kDense, PReLU, ELU, LeakyReLU, Activation, Permute, Conv2DTranspose, Conv1D as kConv1D, BatchNormalization, Add, Concatenate, Multiply, Dropout, merge, Reshape, Flatten, UpSampling1D, Lambda, ZeroPadding1D, Input
from keras import backend as K
from keras.models import Model
import numpy as np
import tqdm
from keras.optimizers import Adam
BATCH_NORM = 'keras'


def Conv1D(filters, kernel_size, strides=1, padding='same', dilation_rate=1, activation=None, momentum=0.9, training=None, BN=True, config=BATCH_NORM,
           use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
           activity_regularizer=None, kernel_constraint=None, bias_constraint=None, dropout=None, name=None, **kwargs):
    """conv -> BN -> activation"""

    def f(x):
        h = x
        if dropout is not None:
            h = Dropout(dropout)(h)
        if padding != "causal++":
            h = kConv1D(filters,
                        kernel_size,
                        strides=strides,
                        padding=padding,
                        dilation_rate=dilation_rate,
                        activation=None,
                        use_bias=use_bias,
                        kernel_initializer=kernel_initializer,
                        bias_initializer=bias_initializer,
                        kernel_regularizer=kernel_regularizer,
                        bias_regularizer=bias_regularizer,
                        activity_regularizer=activity_regularizer,
                        kernel_constraint=kernel_constraint,
                        bias_constraint=bias_constraint,
                        **kwargs)(h)
        else:
            h = ZeroPadding1D(padding=(2, 0))(x)
            h = kConv1D(filters,
                        kernel_size,
                        strides=strides,
                        padding=None,
                        activation=None,
                        use_bias=use_bias,
                        dilation_rate=dilation_rate,
                        kernel_initializer=kernel_initializer,
                        bias_initializer=bias_initializer,
                        kernel_regularizer=kernel_regularizer,
                        bias_regularizer=bias_regularizer,
                        activity_regularizer=activity_regularizer,
                        kernel_constraint=kernel_constraint,
                        bias_constraint=bias_constraint,
                        **kwargs)(h)
            h = Lambda(lambda x_: x_[:, :-2, :])(h)
        h = _activation(activation, BN=BN, name=name, momentum=momentum, training=training, config=config)(h)
        return h

    return f


# DENSE
def Dense(n_units, activation=None, BN=False, channel=1, training=None, config=BATCH_NORM):
    def f(x):
        if len(x._keras_shape[1:]) == 2:
            if channel == 2:
                h = kDense(n_units)(x)
            elif channel == 1:
                h = Permute((2, 1))(kDense(n_units)(Permute((2, 1))(x)))
            else:
                raise ValueError('channel should be either 1 or 2')
            h = _activation(activation, BN=BN, training=training, config=config)(h)
            return h
        elif len(x._keras_shape[1:]) == 1:
            h = kDense(n_units)(x)
            return _activation(activation, BN=BN, training=training, config=config)(h)
        else:
            raise ValueError('len(x._keras_shape) should be either 2 or 3 (including the batch dim)')

    return f


def Sum(axis):
    def f(x):
        return Lambda(lambda x_: K.sum(x_, axis))(x)
    return f


def IsNonZero():
    def f(x):
        return Lambda(lambda x_: K.cast(x_ > 0, np.float32))(x)
    return f


# BatchNorm
def BatchNorm(momentum=0.99, training=True):
    def batchnorm(x, momentum=momentum, training=training):
        return tf.layers.batch_normalization(x, momentum=momentum, training=training)

    def f(x):
        return Lambda(batchnorm, output_shape=tuple([xx for xx in x._keras_shape if xx is not None]))(x)

    return f


# ACTIVATION
def _activation(activation, BN=True, name=None, momentum=0.9, training=None, config=BATCH_NORM):
    """
    A more general activation function, allowing to use just string (for prelu, leakyrelu and elu) and to add BN before applying the activation
    :param training: if using a tensorflow optimizer, training should be K.learning_phase()
                     if using a Keras optimizer, just let it to None
    """

    def f(x):
        if BN and activation != 'selu':
            if config == 'keras':
                h = BatchNormalization(momentum=momentum)(x, training=training)
            elif config == 'tf' or config == 'tensorflow':
                h = BatchNorm(is_training=training)(x)
            else:
                raise ValueError('config should be either `keras`, `tf` or `tensorflow`')
        else:
            h = x
        if activation is None:
            return h
        if activation in ['prelu', 'leakyrelu', 'elu', 'selu']:
            if activation == 'prelu':
                return PReLU(name=name)(h)
            if activation == 'leakyrelu':
                return LeakyReLU(name=name)(h)
            if activation == 'elu':
                return ELU(name=name)(h)
            if activation == 'selu':
                return Selu()(h)
        else:
            h = Activation(activation, name=name)(h)
            return h

    return f


def _selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * K.elu(x, alpha)


def Selu():
    def f(x):
        return Lambda(_selu)(x)
    return f

Using TensorFlow backend.


In [2]:



def get_Q_and_PI_networks(hidden_dim=10, n_actions=14):
    hand = Input((13,4))  # zero everywhere and 1 for your cards
    board = Input((3,13,4))  # 3 rounds of board: flop [0,:,:], turn [1,:,:] and river [2,:,:]
    pot = Input((1,))
    stack = Input((1,))
    opponent_stack = Input((1,))
    blinds = Input((2,))  # small, big blinds
    dealer = Input((1,))  # 1 if you are the dealer, 0 otherwise
    # opponent_model = Input((2,))  # tendency to raise, number of hands played: 2 numbers between 0 and 1
    preflop_plays = Input((6, 5, 2))  # 6 plays max (check then 5 times raises), 5 possible actions (check,bet,call,raise,all-in), 2 players
    flop_plays = Input((6, 5, 2))
    turn_plays = Input((6, 5, 2))
    river_plays = Input((6, 5, 2))
    # value_of_hand = Input((2,))  # combination_type, rank_in_this_type

    # Processing board and hand specifically to detect flush and straight
    color_hand = Sum(1)(hand)
    color_board = Sum((1,2))(board)
    kinds_hand_for_ptqf = Sum(2)(hand)
    kinds_hand_for_straight = IsNonZero()(kinds_hand_for_ptqf)
    kinds_board_for_ptqf = Sum((1,3))(board)
    kinds_board_for_straight = IsNonZero()(kinds_board_for_ptqf)

    colors = Concatenate(2)([Reshape((4,1))(color_hand), Reshape((4,1))(color_board)])
    kinds_for_straight = Concatenate(2)([Reshape((13,1))(kinds_hand_for_straight), Reshape((13,1))(kinds_board_for_straight)])
    kinds_for_ptqf = Flatten()(Concatenate(2)([Reshape((13,1))(kinds_hand_for_ptqf), Reshape((13,1))(kinds_board_for_ptqf)]))
    kinds_for_straight = Conv1D(5,1, activation="selu", BN=False)(kinds_for_straight)
    kinds_for_straight = Conv1D(1, 1, activation='selu', BN=False)(
        Concatenate(-1)([
            Conv1D(1, 3, activation='selu', BN=False)(kinds_for_straight),
            Conv1D(3, 3, dilation_rate=2, activation='selu', BN=False)(kinds_for_straight)
        ])
    )
    kinds_for_straight = Dense(hidden_dim, activation='selu', BN=False)(Flatten()(kinds_for_straight))

    kinds_for_ptqf = Dense(hidden_dim, activation='selu', BN=False)(kinds_for_ptqf)
    kinds_for_ptqf = Dense(hidden_dim, activation='selu', BN=False)(kinds_for_ptqf)
    colors = Conv1D(1, 1, activation='selu', BN=False)(colors)
    colors = Dense(hidden_dim, activation='selu', BN=False)(Flatten()(colors))

    # Process board only
    flop_alone = Dense(hidden_dim, activation='selu', BN=False)(Lambda(lambda x: x[:, 0, :, :])(board))
    flop_alone = Dense(hidden_dim, activation='selu', BN=False)(flop_alone)
    turn_alone = Dense(hidden_dim, activation='selu', BN=False)(Lambda(lambda x: x[:, 1, :, :])(board))
    turn_alone = Dense(hidden_dim, activation='selu', BN=False)(turn_alone)
    river_alone = Dense(hidden_dim, activation='selu', BN=False)(Lambda(lambda x: x[:, 2, :, :])(board))
    river_alone = Dense(hidden_dim, activation='selu', BN=False)(river_alone)

    board_alone = Dense(hidden_dim, activation='selu', BN=False)(Flatten()(Concatenate()([flop_alone, turn_alone, river_alone])))
    board_alone = Dense(hidden_dim, activation='selu', BN=False)(board_alone)

    # Process board and hand together
    bh = Dense(hidden_dim, activation='selu', BN=False)(Concatenate()([Flatten()(board), Flatten()(hand)]))
    bh = Dense(hidden_dim, activation='selu', BN=False)(bh)
    bh = Dense(hidden_dim, activation='selu', BN=False)(Concatenate()([kinds_for_ptqf, kinds_for_straight, colors, board_alone, bh]))
    bh = Dense(hidden_dim, activation='selu', BN=False)(bh)

    n_combination = 9
    probabilities_of_each_combination_board_only = Dense(n_combination, activation='softmax', BN=False)(bh)
    probabilities_of_each_combination_board_hand = Dense(n_combination, activation='softmax', BN=False)(bh)
    board_value = Dense(1, activation='sigmoid', BN=False)(bh)
    board_hand_value = Dense(1, activation='sigmoid', BN=False)(bh)

    # Add pot, blind, dealer, stacks
    pbds = Dense(hidden_dim, activation='selu')(Concatenate()([pot, blinds, dealer, stack, opponent_stack]))
    pbds = Dense(hidden_dim, activation='selu')(pbds)


    # Process plays
    processed_preflop = Dense(hidden_dim, activation='selu')(Flatten()(preflop_plays))
    processed_flop = Dense(hidden_dim, activation='selu')(Concatenate()([Flatten()(flop_plays), Flatten()(flop_alone)]))
    processed_turn = Dense(hidden_dim, activation='selu')(Concatenate()([Flatten()(turn_plays), Flatten()(flop_alone), Flatten()(turn_alone)]))
    processed_river = Dense(hidden_dim, activation='selu')(Concatenate()([Flatten()(river_plays), Flatten()(flop_alone), Flatten()(turn_alone), Flatten()(river_alone)]))
    # processed_opponent = Dense(hidden_dim, activation='prelu', BN=True)(opponent_model)
    plays = Dense(hidden_dim, activation='selu')(Add()([processed_preflop,
                                                        processed_flop,
                                                        processed_turn,
                                                        processed_river,
    #                                                     processed_opponent
                                                       ]))
    plays = Dense(hidden_dim, activation='selu')(plays)

    situation_with_opponent = Dense(hidden_dim, activation='selu')(Concatenate()([plays, pbds, bh]))
    situation_with_opponent = Dense(hidden_dim, activation='selu')(situation_with_opponent)
    Q_values = Dense(n_actions, activation=None)(situation_with_opponent)

    Q = Model([hand, board, pot, stack, opponent_stack, blinds, dealer, preflop_plays, flop_plays, turn_plays, river_plays], [Q_values, probabilities_of_each_combination_board_only, probabilities_of_each_combination_board_hand, board_value, board_hand_value])
    Q.compile(Adam(), ['mse', 'categorical_crossentropy', 'categorical_crossentropy', 'binary_crossentropy', 'binary_crossentropy'])

    situation_with_opponent_pi = Dense(hidden_dim, activation='selu')(Concatenate()([plays, pbds, bh]))
    situation_with_opponent_pi = Dense(hidden_dim, activation='selu')(situation_with_opponent_pi)
    PI_values = Dense(n_actions, activation='softmax')(situation_with_opponent_pi)
    PI = Model([hand, board, pot, stack, opponent_stack, blinds, dealer, preflop_plays, flop_plays, turn_plays, river_plays],
               [PI_values, probabilities_of_each_combination_board_only, probabilities_of_each_combination_board_hand, board_value, board_hand_value])
    PI.compile(Adam(), ['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'binary_crossentropy', 'binary_crossentropy'])
    return Q, PI

In [3]:
Q, PI = get_Q_and_PI_networks(hidden_dim=10, n_actions=16)

In [4]:
from keras.utils import plot_model
plot_model(Q, to_file='Q.png')

In [5]:
import graphviz

In [6]:
print(pydot.find_graphviz())

NameError: name 'pydot' is not defined